In [1]:
import logging
import random
from collections import namedtuple
from itertools import accumulate
from operator import xor
from copy import deepcopy
from typing import Callable
import math

In [2]:
logging.basicConfig(level=logging.INFO)
Nimply = namedtuple("Nimply", "row, num_objects")

In [3]:
class Nim():
    '''a basic class for the game of Nim'''
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i*2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, row: int, num_objects: int) -> None:
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects
        

    def nimming2(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects
            
    def is_terminal(self) -> bool:
        return sum(self._rows) == 0
        
    def utility(self):
        return 1 if self.is_terminal() else 0  

    def actions(self, k = None):
        if k is None:        
            return [(i, j) for i in range(len(self._rows)) for j in range(1, self._rows[i] + 1)]
        else:
            return [(i, j) for i in range(len(self._rows)) for j in range(1, min(self._rows[i] + 1, k + 1))]

In [4]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result

In [5]:
def alphabeta_minmax(state, max_turn, alpha = -1, beta = 1):
    utils_ = utility(state, max_turn)
    if utils_ is not None:
        return utils_

    possible_new_states_list = possible_new_states(state, k)

    child_scores = []
    for new_state in possible_new_states_list:
        score = alphabeta_minmax(new_state, max_turn = not max_turn, alpha=alpha, beta=beta)
        child_scores.append(score)

        if max_turn:
            alpha = max(alpha, score)
        else:
            beta = min(beta, score)
        
        if beta <= alpha:
            break

    if max_turn:
        return max(child_scores)
    else:
        return min(child_scores)

def is_terminal(state):
    
    return sum(state) == 0

def utility(state, max_turn):
    
    if is_terminal(state):
        return 1 if not max_turn else -1

def possible_new_states(state, k):
    
    if k == None:
        
        for row, objects in enumerate(state):            
            for remaining in range(objects):
                yield state[:row] + [remaining,] + state[row + 1 :]
    else:
        
        for row, objects in enumerate(state):            
            for remain in range(objects):
                if objects - remain <= k:
                    yield state[:row] + [remain,] + state[row + 1 :]

def alphabeta_bestmove(state):
    '''select the best move'''
    for new_state in possible_new_states(state, k):
        score = alphabeta_minmax(new_state, False)
        if score > 0:
            break
    
    for _, el in enumerate(state):
        if abs(state[_] - new_state[_]) > 0:
            logging.debug(f"({_},{state[_] - new_state[_]})")
            return (_ , state[_] - new_state[_])